Import dependencies

In [ ]:
%reload_ext autoreload
%autoreload 1
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import cobra

from yeast8model import Yeast8Model

# Construct models of cells of interest

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

# Effect of flux constraints

In [ ]:
from yeast8model import get_ablation_ratio

Convenience functions

In [ ]:
def pool_vs_ablation(ymodel, fractions):
    # get original pool upper bound
    orig_epool = ymodel.model.reactions.get_by_id('prot_pool_exchange').bounds[1]
    
    abl_res_list = []
    prot_pool_exchange_fluxes = []
    for fraction in fractions:
        # impose pool constraint
        ub = fraction*orig_epool
        ymodel.model.reactions.get_by_id('prot_pool_exchange').bounds = (0, ub)
        sol = ymodel.optimize()
        # ablate & ratio
        abl_res = ymodel.ablate()
        # append lists
        abl_res_list.append(abl_res)
        prot_pool_exchange_fluxes.append(sol.fluxes['prot_pool_exchange'])
        
    return abl_res_list, prot_pool_exchange_fluxes

## Parameters

In [ ]:
# Probably doesn't matter if step up or down
fractions = np.linspace(2, 0, num=51)

## Enzyme-constrained

Note: re-load model before re-running.

In [ ]:
# Generate results
abl_list, prot_pool_exchange_fluxes = pool_vs_ablation(wt_ec, fractions)

In [ ]:
# Alternatively, load if saved
with open('ec_pool_vs_ablation.pkl', 'rb') as handle:
    abl_list = pickle.load(handle)

In [ ]:
# Dump data
with open('ec_pool_vs_ablation.pkl', 'wb') as handle:
    pickle.dump(abl_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Plots

In [ ]:
# Define lists for plotting
gr_list = [abl_res.ablated_flux.iloc[0] for abl_res in abl_list]
ratio_list = [get_ablation_ratio(abl_res) for abl_res in abl_list]
abl_flux_list = [abl_res.ablated_flux.iloc[1:].to_list() for abl_res in abl_list]
abl_flux_array = np.array(abl_flux_list)
biomass_components_list = abl_list[0].priority_component.iloc[1:].to_list()

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, prot_pool_exchange_fluxes, 'o-')
ax.set_xlabel(r'Fraction of original $e_{pool}$')
ax.set_ylabel('Protein pool exchange flux')
ax.set_xlim((0,20))
ax.set_ylim((0,2.5))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, gr_list, 'o-')
ax.set_xlabel(r'Fraction of original $e_{pool}$')
ax.set_ylabel('Growth rate')
ax.set_xlim((0,2))
ax.set_ylim((0,1))

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, abl_flux_array, 'o-')
ax.set_xlabel(r'Fraction of original $e_{pool}$')
ax.set_ylabel('Ablated flux')
ax.set_xlim((0,2))
ax.set_ylim((0,4))
ax.legend(biomass_components_list)

In [ ]:
fig, ax = plt.subplots()
ax.plot(fractions, ratio_list, 'o-')
ax.set_xlabel(r'Fraction of original $e_{pool}$')
ax.set_ylabel('Ratio')
ax.set_xlim((0,2))
ax.set_ylim((0.65,0.77))